In [148]:
from keras.layers.convolutional import Convolution2D
from keras.layers import Input, Lambda, Dense
from tensorflow.contrib.layers import convolution2d, fully_connected
from tensorflow.contrib.distributions import kl, Multinomial
from scipy.stats import entropy
import tensorflow as tf
from keras import backend as K
import numpy as np

d = 4

sent_0 = tf.placeholder(shape=(None,d), dtype='float64', name='sent_input_0')
sent_1 = tf.placeholder(shape=(None,d), dtype='float64', name='sent_input_1')
p_ = tf.placeholder(shape=(1,5), dtype='float64', name='target')

sentences = [sent_0, sent_1]

def comU1(vec0, vec1):
    cos_dist = K.sum(vec0 * vec1)/K.sqrt(K.sum(vec0 ** 2))/K.sqrt(K.sum(vec1 ** 2))
    l2_dist = K.sqrt(K.sum(K.square(vec0 - vec1)))
    l1_dist = K.sum(K.abs(vec0 - vec1))

    result = tf.pack([cos_dist, l2_dist, l1_dist])
    
    return result

def comU2(vec0, vec1):
    cos_dist = K.sum(vec0 * vec1)/K.sqrt(K.sum(vec0 ** 2))/K.sqrt(K.sum(vec1 ** 2))
    l2_dist = K.sqrt(K.sum(K.square(vec0 - vec1)))
    result = tf.pack([cos_dist, l2_dist])
    
    return result

In [149]:
from scipy.spatial.distance import cdist
from keras import backend as K
import tensorflow as tf

def input_layer(sentences):
    # Dimensions of two sentence embeddings are supposed to be same
    ds = [sentence.get_shape()[1].value for sentence in sentences]
    assert ds[0] == ds[1]
    
    # norm2 by row
    norm2 = lambda X : K.expand_dims(K.sqrt(K.sum(X ** 2, 1)))
    # Cosine distance by its best definition
    cosine = lambda X, Y : K.dot(X, K.transpose(Y))/norm2(X)/K.transpose(norm2(Y))

    D = cosine(sentences[0], sentences[1])
    
    # Attention weight vectors for sentence 0 and 1
    As = [K.softmax(K.expand_dims(K.sum(D,i),1)) for i in [1,0]]
    
    # Concatenating the original sentence embedding to the cosine similarity, 
    # force the dimension to be expanded from d to 2*d
    atten_embeds = []
    for i in xrange(2):
        atten_embed = tf.concat(1, [sentences[i], As[i] * sentences[i]])
        atten_embed = K.expand_dims(atten_embed, 0)
        atten_embed = K.expand_dims(atten_embed, 3)
        atten_embeds.append(atten_embed)

    return atten_embeds

atten_embeds = input_layer(sentences)

In [155]:
wss = [1,2,3]

# Let the number of filters be 
num_filters = 2*2*d

def sentence_algo1(atten_embeds):
    fea_A = []
    with tf.variable_scope("algo_1"):
        for i, pooling in enumerate([K.max, K.min, K.mean]):
            regM0, regM1 = [], []
            for j, ws in enumerate(wss):
                for k, atten_embed in enumerate(atten_embeds):
                    # Building block A
                    conv = convolution2d(atten_embed, num_filters, kernel_size=[ws, 2*d], stride=[1,1], padding='VALID')
                    conv = tf.squeeze(conv, axis=[0,2])
                    if k == 0:
                        regM0.append(pooling(conv, 0))
                    else:
                        regM1.append(pooling(conv, 0))
            regM0, regM1 = tf.pack(regM0), tf.pack(regM1)

            for n in xrange(num_filters):
                fea_A.append(comU2(regM0[:,n], regM1[:,n]))

        fea_A = K.expand_dims(K.flatten(fea_A),0)
    
    return fea_A

fea_A = sentence_algo1(atten_embeds)

In [153]:
fea_h = sentence_algo1(atten_embeds)

In [145]:
tf.reset_default_graph()

In [156]:
for v in tf.global_variables():
    print v.name

Conv/weights:0
Conv/biases:0
Conv_1/weights:0
Conv_1/biases:0
Conv_2/weights:0
Conv_2/biases:0
Conv_3/weights:0
Conv_3/biases:0
Conv_4/weights:0
Conv_4/biases:0
Conv_5/weights:0
Conv_5/biases:0
Conv_6/weights:0
Conv_6/biases:0
Conv_7/weights:0
Conv_7/biases:0
Conv_8/weights:0
Conv_8/biases:0
Conv_9/weights:0
Conv_9/biases:0
Conv_10/weights:0
Conv_10/biases:0
Conv_11/weights:0
Conv_11/biases:0
Conv_12/weights:0
Conv_12/biases:0
Conv_13/weights:0
Conv_13/biases:0
Conv_14/weights:0
Conv_14/biases:0
Conv_15/weights:0
Conv_15/biases:0
Conv_16/weights:0
Conv_16/biases:0
Conv_17/weights:0
Conv_17/biases:0
Conv_18/weights:0
Conv_18/biases:0
Conv_19/weights:0
Conv_19/biases:0
Conv_20/weights:0
Conv_20/biases:0
Conv_21/weights:0
Conv_21/biases:0
Conv_22/weights:0
Conv_22/biases:0
Conv_23/weights:0
Conv_23/biases:0
Conv_24/weights:0
Conv_24/biases:0
Conv_25/weights:0
Conv_25/biases:0
Conv_26/weights:0
Conv_26/biases:0
Conv_27/weights:0
Conv_27/biases:0
Conv_28/weights:0
Conv_28/biases:0
Conv_29/w

In [133]:
for i in tf.global_variables():
    if "algo_1" in i.name:
        print i.name

In [172]:
[i.name for i in tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES, scope="algo_1")]

[]

In [165]:
g = tf.Graph()

In [189]:
temp = convolution2d(atten_embeds[0], 100, kernel_size=[1, 2*4], stride=[1,1], padding='VALID', \
        weights_regularizer=regularizer, biases_regularizer=regularizer)

In [190]:
temp

<tf.Tensor 'Conv_3/Relu:0' shape=(1, ?, 1, 100) dtype=float64>

In [184]:
regularizer = tf.contrib.layers.l2_regularizer(0.01)

In [33]:
def building_block_B(input, ws, num_filters):
    # Dimension where we want to iteration through with multiple 1D CNN
    dimension = input.get_shape()[2].value

    # Stores the 1d conv output
    convs = []
    # Per dimension iteration
    for d in xrange(dimension):
        conv = convolution2d(tf.expand_dims(input[:,:,d,:],1), num_filters, kernel_size=[1,ws], stride=[1,1], padding='VALID')
        # Removing the dimension with 1
        conv = tf.squeeze(conv, axis=[0,1])
        convs.append(conv)

    return convs

def building_block_A(input, ws, d, num_filters):
    conv = convolution2d(input, num_filters, kernel_size=[ws, 2*d], stride=[1,1], padding='VALID')
    return conv

In [105]:
def sentence_algo2(atten_embeds, num_filters_A, num_filters_B):
    fea_a, fea_b = [], []
    for i, pooling in enumerate([K.max, K.min, K.mean]):
        atten_embed_0, atten_embed_1 = atten_embeds

        # Working with building block A, moving the window across the whole length of the word embedding
        for j_0, ws_0 in enumerate(wss):
            oG0A = building_block_A(atten_embed_0, ws_0, d, num_filters_A)
            for j_1, ws_1 in enumerate(wss):
                oG1A = building_block_A(atten_embed_1, ws_1, d, num_filters_A)
                fea_a.append(comU1(oG0A, oG1A))

        # Working with building block B, the per dimensional CNN
        for b, ws in enumerate(wss):
            oG0B = building_block_B(atten_embed_0, ws, num_filters_B)
            oG0B = tf.pack([pooling(conv,0) for conv in oG0B])

            oG1B = building_block_B(atten_embed_1, ws, num_filters_B)
            oG1B = tf.pack([pooling(conv,0) for conv in oG1B])
            
            for n in xrange(num_filters_B):
                fea_b.append(comU1(oG0B[:,n], oG1B[:,n]))
        
    fea_v = tf.concat(0, map(K.flatten, [fea_a, fea_b]))
    
    return fea_v

In [113]:
class SentenceModeling():
    def __init__(conf, atten_embeds):
        self.conf = conf
        self.atten_embeds = atten_embeds
        self.fea_h, fea_v = None, None

    # Algorithm 1 Horizontal Comparison
    def horizontal_comparison(self):
        if self.fea_h is not None:
            return self.fea_h

        fea_h = []
        with tf.variable_scope("algo_1"):
            for i, pooling in enumerate([K.max, K.min, K.mean]):
                regM0, regM1 = [], []
                for j, ws in enumerate(wss):
                    for k, atten_embed in enumerate(atten_embeds):
                        # Working with building block A, moving the window across the whole length of the word embedding
                        conv = building_block_A(atten_embed, ws_0, d, num_filters_A)
                        conv = tf.squeeze(conv, axis=[0,2])
                        if k == 0:
                            regM0.append(pooling(conv, 0))
                        else:
                            regM1.append(pooling(conv, 0))
                regM0, regM1 = tf.pack(regM0), tf.pack(regM1)

                for n in xrange(num_filters):
                    fea_h.append(comU2(regM0[:,n], regM1[:,n]))

            fea_h = K.expand_dims(K.flatten(fea_h),0)

            self.fea_h = fea_h
        
        return fea_h

    # Algorithm 2 Vertical Comparison
    def vertical_comparison(self):
        if self.fea_v is not None:
            return self.fea_v

        fea_a, fea_b = [], []

        with tf.variable_scope("algo_2"):
            for i, pooling in enumerate([K.max, K.min, K.mean]):
                atten_embed_0, atten_embed_1 = self.atten_embeds

                # Working with building block A, moving the window across the whole length of the word embedding
                for j_0, ws_0 in enumerate(wss):
                    oG0A = building_block_A(atten_embed_0, ws_0, d, self.conf.num_filters_A)
                    for j_1, ws_1 in enumerate(wss):
                        oG1A = building_block_A(atten_embed_1, ws_1, d, self.conf.num_filters_A)
                        fea_a.append(comU1(oG0A, oG1A))

                # Working with building block B, the per dimensional CNN
                for b, ws in enumerate(wss):
                    oG0B = building_block_B(atten_embed_0, ws, self.conf.num_filters_B)
                    oG0B = tf.pack([pooling(conv,0) for conv in oG0B])

                    oG1B = building_block_B(atten_embed_1, ws, self.conf.num_filters_B)
                    oG1B = tf.pack([pooling(conv,0) for conv in oG1B])
                    
                    for n in xrange(num_filters_B):
                        fea_b.append(comU1(oG0B[:,n], oG1B[:,n]))
            
            # Concatenate them up! Oops that's a lot...
            fea_v = K.expand_dims(tf.concat(0, map(K.flatten, [fea_a, fea_b])),0)

            self.fea_a, self.fea_b, self.fea_v = fea_a, fea_b, fea_v
        
        return fea_v

In [114]:
# Algorithm 1 Horizontal Comparison
def horizontal_comparison(self):
    if self.fea_h is not None:
        return self.fea_h

    fea_h = []
    with tf.variable_scope("algo_1"):
        for i, pooling in enumerate([K.max, K.min, K.mean]):
            regM0, regM1 = [], []
            for j, ws in enumerate(wss):
                for k, atten_embed in enumerate(atten_embeds):
                    # Working with building block A, moving the window across the whole length of the word embedding
                    conv = building_block_A(atten_embed, ws_0, d, num_filters_A)
                    conv = tf.squeeze(conv, axis=[0,2])
                    if k == 0:
                        regM0.append(pooling(conv, 0))
                    else:
                        regM1.append(pooling(conv, 0))
            regM0, regM1 = tf.pack(regM0), tf.pack(regM1)

            for n in xrange(num_filters):
                fea_h.append(comU2(regM0[:,n], regM1[:,n]))

        fea_h = K.expand_dims(K.flatten(fea_h),0)

        self.fea_h = fea_h
        
        return fea_h
fea_h = setence_model.horizontal_comparison()

NameError: name 'conf' is not defined

In [107]:
fea_v

<tf.Tensor 'concat_8:0' shape=(486,) dtype=float64>

In [101]:
fea_a, fea_b = map(K.flatten, [fea_a, fea_b])

In [103]:
a, b = map(K.flatten, [fea_a, fea_b])

In [104]:
a

<tf.Tensor 'Reshape_15:0' shape=(81,) dtype=float64>

In [75]:
n_hidden = 5 
p = fully_connected(fully_connected(fea_A, n_hidden, activation_fn=K.tanh), 5, activation_fn=tf.nn.log_softmax)
# p_ = sparse_target_distribution(y)

cross_entropy = -tf.reduce_sum(p_ * tf.log(p))
entropy = -tf.reduce_sum(p_ * tf.log(p_ + 0.00001))
kl_divergence = cross_entropy - entropy

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(kl_divergence)

In [42]:
p

NameError: name 'p' is not defined

In [ ]:
conv_layers_A[0].get_weights()[0].shape

In [48]:
def sparse_target_distribution(y):
    y_floor = int(np.floor(y))
    p = np.zeros(5)
    for i in xrange(y):
        if i == y_floor:
            p[i] = y - y_floor
        elif i == y_floor - 1:
            p[i] = y - y_floor + 1
    
    return p

In [62]:
Multinomial(logits=p_)

TypeError: __init__() takes at least 2 arguments (2 given)

In [63]:
p = [.2, .3, .5]
Multinomial(n=4., p=p)

In [ ]:
for sentences in range(sentence_pairs):
    sent_0, sent_1, target = sentences
    
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [ ]:
def initialize_block_A(d, wss, activation=K.tanh):    
    conv_layers = [Convolution2D(num_filters, ws, 2*d, dim_ordering="tf", activation=activation) for ws in wss]
    
    return conv_layers

def initialize_block_B(d, wss, activation=K.tanh):    
    conv_layers = [Convolution2D(num_filters, 1, ws, dim_ordering="tf", activation=activation) for ws in wss]
    
    return conv_layers

def initialize_dense(n_hidden=150, activation=K.tanh):
    dense = Dense(n_hidden, activation=activation)
    softmax = Dense(5, activation=K.softmax)
    
    return (dense, softmax)

In [ ]:
output

In [ ]:
x

In [ ]:
K.transpose(A_0 * sent_0)